In [ ]:
import ee
import pandas as pd
import numpy as np
import time
from operator import itemgetter
from datetime import datetime, timedelta
from shapely.geometry import Polygon

import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

!pip install -q geopandas
import geopandas as gpd

     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 15.4 MB 46.9 MB/s 


In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=k-8kypr4lI4EMKa7rEBAIVYgD0JzpvNWe-1iI8qPE5A&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWibwqfo8wEB_z4h0bQ8P-joq_KZL8AbDRIq53BarP2KDNiZs_zaKFM

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def convertUnix(x):
    """
    Converts EE.Date to Y-M-D formst
    """
    x = x.getInfo()["value"]
    return datetime.utcfromtimestamp(x/1000).strftime("%Y-%m-%d")# %H:%M:%S')


##### ADJUST TO BETTER DATES/INTERVALS (PRE+POST FIRE) #####
def dateSequence(start, end, dateInterval=3, shift=-9):
    """
    Returns a list of date pairs between a start/end date.
    
    ###
    dateInterval: adjust number of days between dates 
    shift: adjust start date to account for pre-fire conditions
    """
    start = datetime.fromisoformat(start) + timedelta(days=shift)
    end = datetime.fromisoformat(end)
    
    dateSeq = []
    while end-start > timedelta(days=0):
        dateSeq.append(start.isoformat()[:10])
        start += timedelta(days=dateInterval)

    dateSeq.append(end.isoformat()[:10])
    
    # Returns list of date sequence pairs    
    return [[dateSeq[i], dateSeq[i+1]] for i in range(len(dateSeq)-1)]


def boundsBuffer(x, buffer=0.2):
    """
    Returns a Polygon geometry that represents a bounding box with a default 20% buffer
    """
    minx, miny, maxx, maxy = x
    buffer_x, buffer_y = np.abs(buffer*(maxx-minx)), np.abs(buffer*(maxy-miny))
    minx -= buffer_x
    maxx += buffer_x
    miny -= buffer_y
    maxy += buffer_y
    
    coords = ((minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny))
    return Polygon(coords)


def reduceImage(image):
    """
    Applies a reducer over an image + ee.Feature geometry and returns ee.Dictionary
    """
    image = ee.Image(image)
    fireName, start, end = image.get("FIRE_NAME"), image.get("start"), image.get("end") 
    feature = ee.Feature(fires.filter(ee.Filter.eq("FIRE_NAME",
                                                    image.get("FIRE_NAME"))
                                        ).first())
    
    reduction = image.reduceRegion(reducer=ee.Reducer.mean(),
                                   geometry=feature.geometry(),
                                   scale=4638.3,
                                   maxPixels=1e10,
                                   tileScale=4)
    
    # Add additional keys
    reduction = reduction.set("FIRE_NAME", fireName)
    reduction = reduction.set("start", start)
    reduction = reduction.set("end", end)
    
    return reduction

In [ ]:
# California wildfires 2015-2019
firesDf = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/caFirePerimeters_2015_2019.geojson")

# Fires < 10000 acres or without a listed "contain date" are dropped
firesDf = firesDf[(firesDf["GIS_ACRES"] >= 10000) & (~firesDf["CONT_DATE"].isna())]
firesDf["GIS_ACRES"] = firesDf["GIS_ACRES"].round(2)

demo = firesDf.bounds.apply(lambda x: x[3]>sfLowerBound, axis=1)
firesDf = firesDf[demo].reset_index(drop=True)


# Convert fire geometry to bounding box with 10% buffer
firesDf["geometry"] = firesDf["geometry"].bounds.apply(boundsBuffer, axis=1)

print(firesDf.shape)
firesDf.head()

In [ ]:
CA_bounds = ee.FeatureCollection("FAO/GAUL/2015/level1"
             ).filter(ee.Filter.eq("ADM0_NAME", "United States of America")
             ).filter(ee.Filter.eq("ADM1_NAME", "California"))


gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET"
           ).filterDate("2018-01-01", "2021-01-01"
           ).map(lambda x: x.clip(CA_bounds))


# Keys to select from dictionary
dfKeys = ["FIRE_NAME", "start", "end", "pr_mean", "rmax_mean", "rmin_mean",
          "sph_mean", "srad_mean", "th_mean", "tmmn_mean", "tmmx_mean", "vs_mean",
          "erc_mean", "eto_mean", "bi_mean", "fm100_mean", "fm1000_mean", "etr_mean", "vpd_mean"]


# Date ranges to reduce for each fire
dateSequences = {}
for fireName, startDate, endDate in df[["FIRE_NAME", "ALARM_DATE", "CONT_DATE"]].values:
    dateSequences[fireName] = dateSequence(startDate, endDate)

In [ ]:
totalTime = time.time()
chunks = list(range(0, len(df)+1, 5))
lst = []


# Reduce chunks of 5 fires at a time
for i in range(len(chunks)-1):     
    startTime = time.time()
    dfChunk = df.iloc[chunks[i]:chunks[i+1],:]         # Subset gpd dataframe into chunks
    fires = geemap.gdf_to_ee(dfChunk, geodesic=True)   # Push to EE as a FeatureCollection
    
    fireDates = [[j, dateSequences[j]] for j in dfChunk["FIRE_NAME"].to_list()]

    # Gridmet images filtered by date to reduce over
    fireImages = ee.List([])
    for fire, dateRange in fireDates:
        for start, end in dateRange:
            fireImages = fireImages.add(gridmet.filterDate(start, end
                                              ).reduce(ee.Reducer.mean()
                                              ).set("FIRE_NAME", fire,
                                                    "start", start,
                                                    "end", end))
            
    # Maps image reducer over ee.List    
    results = fireImages.map(reduceImage).getInfo()
    
    # Omits reductions with no gridmet data
    for k in results:
        try:
            lst += [list(itemgetter(*dfKeys)(k))]
        except KeyError:  
            pass
        
    print("Chunk {} Runtime {} minutes".format(i+1, np.round((time.time()-startTime)/60, 3)))

print("Total Runtime {} minutes".format(np.round((time.time()-totalTime)/60, 3)))

Chunk 1 Runtime 0.01 minutes
Chunk 2 Runtime 0.063 minutes
Chunk 3 Runtime 0.008 minutes
Chunk 4 Runtime 0.01 minutes
Chunk 5 Runtime 0.016 minutes
Chunk 6 Runtime 0.011 minutes
Chunk 7 Runtime 0.013 minutes
Chunk 8 Runtime 0.009 minutes
Chunk 9 Runtime 0.012 minutes
Chunk 10 Runtime 0.007 minutes
Chunk 11 Runtime 0.013 minutes
Chunk 12 Runtime 0.007 minutes
Chunk 13 Runtime 0.012 minutes
Chunk 14 Runtime 0.013 minutes
Total Runtime 0.206 minutes


In [ ]:
fireWeather = pd.DataFrame(lst, columns=dfKeys)
fireWeather.to_csv("/content/drive/Shareddrives/Capstone 2022/Anthony/fireWeather.csv", index=False)

print(fireWeather.shape)
fireWeather.head()

(1274, 19)


,FIRE_NAME,start,end,pr_mean,rmax_mean,rmin_mean,sph_mean,srad_mean,th_mean,tmmn_mean,tmmx_mean,vs_mean,erc_mean,eto_mean,bi_mean,fm100_mean,fm1000_mean,etr_mean,vpd_mean
0,CZU LIGHTNING COMPLEX,2020-08-07,2020-08-10,0.000000,81.676432,29.960919,0.008271,306.576709,276.918947,286.842561,301.396653,2.380347,51.519884,5.326313,36.307465,12.601621,11.709238,6.616621,1.432853
1,CZU LIGHTNING COMPLEX,2020-08-10,2020-08-13,0.000000,78.402716,30.193626,0.008317,306.029268,228.135049,287.603325,301.440461,2.324803,53.281070,5.451737,36.834986,11.457121,11.622644,6.843215,1.469019
2,CZU LIGHTNING COMPLEX,2020-08-13,2020-08-16,0.298388,52.868899,14.019846,0.007454,286.421807,298.470192,292.680710,309.436899,3.242759,56.734205,7.547748,36.051035,8.872244,11.090852,10.615395,3.050329
3,CZU LIGHTNING COMPLEX,2020-08-16,2020-08-19,0.564305,57.950432,20.000848,0.009025,260.613258,313.779679,294.049307,308.495380,6.759483,59.416059,9.288845,57.920990,7.927109,10.769540,13.913555,2.748504
4,CZU LIGHTNING COMPLEX,2020-08-19,2020-08-22,0.000000,75.142782,24.593267,0.008143,293.377476,316.946565,287.894733,303.990709,7.355845,63.100665,8.090542,72.602142,8.278097,10.298148,11.899383,1.827913
